In [ ]:
!pip install modin
!pip install ray
!pip install dask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **Read CSV with Pandas, Dask, Modin/Ray**

In [ ]:
import pandas as pd
import dask.dataframe as dd
import modin.pandas as mpd
import ray
import time

ImportError: ignored

In [ ]:
csvfile='C:\Users\jeeye\Documents\price_paid_records.csv'

Mounted at /content/drive/


In [ ]:
#Pandas to read the CSV

start = time.time()
df=pd.read_csv(csvfile)
end = time.time()
print('Time to read the CSV (pandas): ',end - start, 'seconds')

Time to read the CSV (pandas):  64.92612504959106 seconds


In [ ]:
#Modin/Ray to read the CSV

start = time.time()
ray.shutdown()
ray.init()
mpd.read_csv(csvfile)
end = time.time()
print('Time to read the CSV (modin/ray): ',end - start, 'seconds')

(deploy_ray_func pid=3040) tcmalloc: large alloc 1202847744 bytes == 0x3f06000 @  0x7f575a1ec1e7 0x4a3940 0x5b438c 0x5d0ccd 0x5939af 0x516337 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x4bca8a 0x5134a6 0x549576 0x4bca8a 0x5134a6 0x4bc98a 0x7f5756ec5e02 0x7f5756f59db6 0x7f5756ecc306 0x7f57570406ab 0x7f5756fa188f 0x7f57570687d3 0x7f575706972a 0x7f575707b18e 0x7f5757054530 0x7f5757278f06 0x7f5757225a3e 0x7f5757225c96 0x7f57576bccab 0x7f57576bdee1
(deploy_ray_func pid=3041) tcmalloc: large alloc 1202847744 bytes == 0x4280000 @  0x7f30ef15e1e7 0x4a3940 0x5b438c 0x5d0ccd 0x5939af 0x516337 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x4bca8a 0x5134a6 0x549576 0x4bca8a 0x5134a6 0x4bc98a 0x7f30ebe37e02 0x7f30ebecbdb6 0x7f30ebe3e306 0x7f30ebfb26ab 0x7f30ebf1388f 0x7f30ebfda7d3 0x7f30ebfdb72a 0x7f30ebfed18e 0x7f30ebfc6530 0x7f30ec1eaf06 0x7f30ec197a3e 0x7f30ec197c96 0x7f30ec62ecab 0x7f30ec62fee1


In [ ]:
#Dask to read the CSV

start = time.time()
dd.read_csv(csvfile)
end = time.time()
print('Time to read the CSV (dask): ',end - start, 'seconds')

**Dask took the least time reading the CSV file!**


# **Clean the Column Names**

In [ ]:
# Remove spaces and special chars from the cols

data.columns=data.columns.str.lower()
data.columns = data.columns.str.replace(' ', '')
data.columns=data.columns.str.replace('[^\w]','_',regex=True)
print(data.columns)

Index(['transactionuniqueidentifier', 'price', 'dateoftransfer',
       'propertytype', 'old_new', 'duration', 'town_city', 'district',
       'county', 'ppdcategorytype', 'recordstatus_monthlyfileonly'],
      dtype='object')


# **Validation**



In [ ]:
%%writefile utility.py
import yaml
import logging
import os
import subprocess
import pandas as pd
import re
import gc
import datetime

def read_config_file(filepath):
  with open(filepath, 'r') as stream:
    try:
      return yaml.safe_load(stream)
    except yaml.YAMLerror as exc:
      logging.error(exc)

def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df, table_config):
  df.columns=df.columns.str.lower()
  df.columns=df.columns.str.replace('[^\w]','_',regex=True)
  df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
  df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
  expected_col = list(map(lambda x: x.lower(), table_config['columns']))
  expected_col.sort()
  df.columns = list(map(lambda x: x.lower(), list(df.columns)))
  df=df.reindex(sorted(df.columns), axis=1)
  if len(df.columns) == len(expected_col) and list(expected_col) == list(df.columns):
    print("Column name and column length validation passed")
    return 1
  else:
    print("Column name and column length validation failed")
    mismatched_columns_file = list(set(df.columns).difference(expected_col))
    print("Following file columns are not in the YAML file", mismatched_columns_file)
    missing_YAML_file = list(set(expected_col).difference(df.columns))
    print("Following YAML columns are not in the file uploaded", missing_YAML_file)
    logging.info(f'df columns: {df.columns}')
    logging.info(f'expected columns: {expected_col}')
    return 0

Writing utility.py


In [ ]:
%%writefile store.yaml
file_type: csv
dataset_name: testfile
file_name: price_paid_records
table_name: endsurv
inbound_delimiter: ','
outbound_delimiter: '|'
skip_leading_rows: 1
columns:
  - transaction_unique_identifier
  - price
  - date_of_transfer
  - property_type
  - old_new
  - duration
  - town_city
  - district
  - county
  - ppdcategory_type
  - record_status_monthly_file_only

Writing store.yaml


In [ ]:
# Read config file

import utility as util
config_data = util.read_config_file("store.yaml")

In [ ]:
config_data['inbound_delimiter']

','

In [ ]:
#data of the config file
config_data

{'columns': ['transaction_unique_identifier',
  'price',
  'date_of_transfer',
  'property_type',
  'old_new',
  'duration',
  'town_city',
  'district',
  'county',
  'ppdcategory_type',
  'record_status_monthly_file_only'],
 'dataset_name': 'testfile',
 'file_name': 'price_paid_records',
 'file_type': 'csv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'table_name': 'endsurv'}

In [ ]:
#Read the file using config file
file_type = config_data['file_type']
source_file = "drive/My Drive/Colab Notebooks/" + config_data['file_name'] + f'.{file_type}'
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,Transaction unique identifier,Price,Date of Transfer,Property Type,Old/New,Duration,Town/City,District,County,PPDCategory Type,Record Status - monthly file only
0,{81B82214-7FBC-4129-9F6B-4956B4A663AD},25000,1995-08-18 00:00,T,N,F,OLDHAM,OLDHAM,GREATER MANCHESTER,A,A
1,{8046EC72-1466-42D6-A753-4956BF7CD8A2},42500,1995-08-09 00:00,S,N,F,GRAYS,THURROCK,THURROCK,A,A
2,{278D581A-5BF3-4FCE-AF62-4956D87691E6},45000,1995-06-30 00:00,T,N,F,HIGHBRIDGE,SEDGEMOOR,SOMERSET,A,A
3,{1D861C06-A416-4865-973C-4956DB12CD12},43150,1995-11-24 00:00,T,N,F,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE,A,A
4,{DD8645FD-A815-43A6-A7BA-4956E58F1874},18899,1995-06-23 00:00,S,N,F,WAKEFIELD,LEEDS,WEST YORKSHIRE,A,A


In [ ]:
#Validate the Header of the file
util.col_header_val(df,config_data)

Column name and column length validation passed


1

In [ ]:
print("Columns of files are:" , df.columns)
print("Columns of YAML are:" , config_data['columns'])

Columns of files are: Index(['transaction_unique_identifier', 'price', 'date_of_transfer',
       'property_type', 'old_new', 'duration', 'town_city', 'district',
       'county', 'ppdcategory_type', 'record_status_monthly_file_only'],
      dtype='object')
Columns of YAML are: ['transaction_unique_identifier', 'price', 'date_of_transfer', 'property_type', 'old_new', 'duration', 'town_city', 'district', 'county', 'ppdcategory_type', 'record_status_monthly_file_only']


In [ ]:
if util.col_header_val(df,config_data)==0:
    print("Validation failed")
    print("Columns of the file does not match the YAML")
else:
    print("Column validation passed")
    print('Preview of the data\n', df.head())

Column name and column length validation passed
Column validation passed
Preview of the data
             transaction_unique_identifier  price  date_of_transfer  \
0  {81B82214-7FBC-4129-9F6B-4956B4A663AD}  25000  1995-08-18 00:00   
1  {8046EC72-1466-42D6-A753-4956BF7CD8A2}  42500  1995-08-09 00:00   
2  {278D581A-5BF3-4FCE-AF62-4956D87691E6}  45000  1995-06-30 00:00   
3  {1D861C06-A416-4865-973C-4956DB12CD12}  43150  1995-11-24 00:00   
4  {DD8645FD-A815-43A6-A7BA-4956E58F1874}  18899  1995-06-23 00:00   

  property_type old_new duration   town_city            district  \
0             T       N        F      OLDHAM              OLDHAM   
1             S       N        F       GRAYS            THURROCK   
2             T       N        F  HIGHBRIDGE           SEDGEMOOR   
3             T       N        F     BEDFORD  NORTH BEDFORDSHIRE   
4             S       N        F   WAKEFIELD               LEEDS   

               county ppdcategory_type record_status_monthly_file_only  
0  

# **Save pipe separated file as .gz**

In [ ]:
#csv to gz

import gzip
import csv

df.to_csv('dfgz.csv.gz',
      sep='|',
      header=True,
      index=False,
      quoting=csv.QUOTE_ALL,
      compression='gzip',
      quotechar='"',
      doublequote=True,
      line_terminator='\n')

KeyboardInterrupt: ignored

In [ ]:
# Number of cols in the gz file
import os
entries = os.listdir('./dfgz.csv.gz')
for entry in entries:
    print(entry)

NotADirectoryError: ignored

# **Summary of the File**

In [ ]:
#Size of the CSV file

import os
data=dd.read_csv(csvfile)
print('Size of the CSV file: ', str(os.path.getsize(csvfile), 'Bytes'))

            Transaction unique identifier  Price  Date of Transfer  \
0  {81B82214-7FBC-4129-9F6B-4956B4A663AD}  25000  1995-08-18 00:00   
1  {8046EC72-1466-42D6-A753-4956BF7CD8A2}  42500  1995-08-09 00:00   
2  {278D581A-5BF3-4FCE-AF62-4956D87691E6}  45000  1995-06-30 00:00   
3  {1D861C06-A416-4865-973C-4956DB12CD12}  43150  1995-11-24 00:00   
4  {DD8645FD-A815-43A6-A7BA-4956E58F1874}  18899  1995-06-23 00:00   

  Property Type Old/New Duration   Town/City            District  \
0             T       N        F      OLDHAM              OLDHAM   
1             S       N        F       GRAYS            THURROCK   
2             T       N        F  HIGHBRIDGE           SEDGEMOOR   
3             T       N        F     BEDFORD  NORTH BEDFORDSHIRE   
4             S       N        F   WAKEFIELD               LEEDS   

               County PPDCategory Type Record Status - monthly file only  
0  GREATER MANCHESTER                A                                 A  
1            THURROC

In [ ]:
#Number of rows/cols
print('Number of Rows: ',len(df.rows))
print('Number of columns:', len(df.columns))